In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

In [8]:
dfFlight = spark.read.json('hdfs://namenode:9000/data/flights.json')

In [9]:
df_1 = dfFlight.select('reg_number', 'flight_icao', 'dep_icao', 'arr_icao', 'airline_icao', 'aircraft_icao', 'status')
df_1.show()

+----------+-----------+--------+--------+------------+-------------+--------+
|reg_number|flight_icao|dep_icao|arr_icao|airline_icao|aircraft_icao|  status|
+----------+-----------+--------+--------+------------+-------------+--------+
|  RA-67159|       null|    null|    null|         UVT|         CRJ2|en-route|
|  RA-67169|       null|    OMDW|    null|         RNA|         CL60|en-route|
|  RA-73760|    AFL1231|    UWUU|    UUEE|         AFL|         A320|en-route|
|  RA-73782|     AFL426|    UUEE|    HESH|         AFL|         A333|en-route|
|  RA-73808|     SVR619|    URSS|    USCC|         SVR|         A320|en-route|
|  RA-73809|     SVR627|    UUDD|    UMKK|         SVR|         A320|en-route|
|  RA-73816|     SVR391|    ULLI|    UMKK|         SVR|         A319|en-route|
|  RA-73817|     SVR372|    UMKK|    UUDD|         SVR|         A320|en-route|
|  RA-73824|    SVR9486|    UWKE|    UNNT|         SVR|         A320|en-route|
|  RA-73833|     SVR508|    URSS|    UWKD|         S

In [11]:
dfAirport = spark.read.json('hdfs://namenode:9000/data/airports.json')

In [12]:
df_1 = df_1.join(dfAirport, df_1.dep_icao == dfAirport.icao_code,how='left')

In [13]:
depAirport = df_1.drop('country_code', 'iata_code' , 'icao_code' , 'lat' ,'lng')
depAirport = depAirport.withColumnRenamed("name","dep_airport")
depAirport.show(5)

+----------+-----------+--------+--------+------------+-------------+--------+--------------------+
|reg_number|flight_icao|dep_icao|arr_icao|airline_icao|aircraft_icao|  status|         dep_airport|
+----------+-----------+--------+--------+------------+-------------+--------+--------------------+
|  RA-67159|       null|    null|    null|         UVT|         CRJ2|en-route|                null|
|  RA-67169|       null|    OMDW|    null|         RNA|         CL60|en-route|Dubai World Centr...|
|  RA-73760|    AFL1231|    UWUU|    UUEE|         AFL|         A320|en-route|Ufa International...|
|  RA-73782|     AFL426|    UUEE|    HESH|         AFL|         A333|en-route|Sheremetyevo Inte...|
|  RA-73808|     SVR619|    URSS|    USCC|         SVR|         A320|en-route|Sochi Internation...|
+----------+-----------+--------+--------+------------+-------------+--------+--------------------+
only showing top 5 rows



In [14]:
departuresFlight = depAirport.join(dfAirport, df_1.arr_icao == dfAirport.icao_code,how='left')

In [26]:
departuresFlight = departuresFlight.drop('country_code', 'iata_code' , 'icao_code' , 'lat' ,'lng')
departuresFlight = departuresFlight.withColumnRenamed("name","arr_airport")
departuresFlight.count()
lastDeparturesFlightDf = departuresFlight.na.drop(subset=["arr_airport","dep_airport"])
lastDeparturesFlightDf.show()

+----------+-----------+--------+--------+------------+-------------+--------+--------------------+--------------------+
|reg_number|flight_icao|dep_icao|arr_icao|airline_icao|aircraft_icao|  status|         dep_airport|         arr_airport|
+----------+-----------+--------+--------+------------+-------------+--------+--------------------+--------------------+
|  RA-73760|    AFL1231|    UWUU|    UUEE|         AFL|         A320|en-route|Ufa International...|Sheremetyevo Inte...|
|  RA-73782|     AFL426|    UUEE|    HESH|         AFL|         A333|en-route|Sheremetyevo Inte...|Sharm el-Sheikh I...|
|  RA-73808|     SVR619|    URSS|    USCC|         SVR|         A320|en-route|Sochi Internation...| Chelyabinsk Airport|
|  RA-73809|     SVR627|    UUDD|    UMKK|         SVR|         A320|en-route|Domodedovo Intern...|   Khrabrovo Airport|
|  RA-73816|     SVR391|    ULLI|    UMKK|         SVR|         A319|en-route|     Pulkovo Airport|   Khrabrovo Airport|
|  RA-73817|     SVR372|    UMKK